# Построение движка для игр на pygame - урок 8

## Повторение

Рассмотрим решение домашнего задания

In [2]:
from first_engine.game import Game
from first_engine import objects
import pygame


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=4)
        stand_right = pygame.image.load('./sprites/viking/stand/right/1.png')
        stand_left = pygame.image.load('./sprites/viking/stand/left/1.png')
        run_right = pygame.image.load('./sprites/viking/run/right/1.png')  # добавляем картинку с бегом
        run_left = pygame.image.load('./sprites/viking/run/left/1.png')  # добавляем картинку с бегом
        attack_right = pygame.image.load('./sprites/viking/attack1/right/1.png') # добавляем картинку с атакой
        attack_left = pygame.image.load('./sprites/viking/attack1/left/1.png') # добавляем картинку с атакой
        attack = 0 # объявляем атаку

        side = 'right' # объявляем сторону

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)

                sides = hero.motion_control()  # сохраняем возвращаемые значения

                hero.blit()  # для отладки

                if attack: # если атака есть
                    if side == 'right': # если вправо
                        self.surface.blit(attack_right, hero.body)
                    elif side == 'left': # если влево
                        self.surface.blit(attack_left, hero.body)
                    attack -= 1
                elif sum(sides.values()) == 0 and side == 'right': # если вправо
                    self.surface.blit(stand_right, hero.body)
                elif sum(sides.values()) == 0 and side == 'left': # если влево
                    self.surface.blit(stand_left, hero.body)
                elif sides['right']:
                    side = 'right'
                    self.surface.blit(run_right, hero.body)
                elif sides['left']:
                    side = 'left'
                    self.surface.blit(run_left, hero.body)

            for event in self.events():
                # print(event)  # отслеживание событий
                if event.type == 768 and event.key == 102: # если нажата кнопка F
                    attack = 15 # объявляем число кадров атаки
                self.close(event)


FirstGame(width=600, height=400).run()

## Внутренняя анимация

Давайте попробуем добавить анимацию внутри одного действия персонажа

In [3]:
from first_engine.game import Game
from first_engine import objects
import pygame
import os


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=8)

        stand = './sprites/viking/stand/right/' # возьмем путь до всех спрайтов в одной папке
        stand = [pygame.image.load(stand + i) for i in os.listdir(stand)] # сделаем список загруженных спрайтов

        time_to_update_sprites = 6 # назначаим время до смены спрайта
        sprite_id = 0 # начинаем с 1го спрайта в каждом списке

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)

                sides = hero.motion_control()
                
                time_to_update_sprites -= 1     # запускаяем обратный отсчет до обновления спрайта
                if time_to_update_sprites == 0: # если время до обновления = 0
                    sprite_id += 1              # тогда переключаемся на следующий спрайт
                    time_to_update_sprites = 6  # обновляем счетчик до обновления
                    if sprite_id == len(stand): # если id спрайта равна длине спискаспрайтов
                        sprite_id = 0           # начинаем снова с первого спрайта

                if sum(sides.values()) == 0: # если не нажата ни одна кнопка движения
                    self.surface.blit(stand[sprite_id], hero.body) # тогда отрисовываем спрайты из группы stand по очереди

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=600, height=400).run()

Давайте допишем нашу функцию __load_sprites__ в классе __Objects__

In [ ]:
class Object:
    sprites = {} # добавим словарь для хранения спрайтов и их боксов

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255)):...

    def load_sprites(self, name: str, path: str) -> list[pygame.Surface]: 
        """
        Метод для загрузки скинов и созданния тел
        :param name: str - имя под которым будем хранить загруженные скины и созданные тела
        :param path: str - путь до папки с файлами
        :return skin_body: dict - возвращаем только что загруженные скины и созданные тела
        """
        # это для удобства, что бы не нужно было думать - поставили мы /, в конце пути или нет
        path = path if path[-1] == '/' else path + '/'

        skin = [pygame.image.load(path + i) for i in os.listdir(path)] # загружаем сразу картинки
        self.sprites[name] = skin # потом добавляем этот дикт в dict класса по заданному нами имени
        return skin # это тоже для удобства - возвращаем только что закруженные спрайты и созданные боксы

    def blit(self):...

    def recolor(self, color):...

    def replace(self, x=None, y=None):...

Теперь нам надо подумать, как применить, этот dict, который теперь хранит в себе кадры для разных действий, собственно к самим действиям.

Давайте для начала, сделаем еще 1 метод __remake_for_skin__ в классе __Object__ для того, что бы переделывать скин и тело.

In [ ]:
class Object:
    sprites = {}
    last_action = 'right' # создадим переменную последнего действия, в будущем она нам очень поможет

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255)):...

    def load_sprites(self, name: str, path: str) -> list[pygame.Surface]:...

    def remake_for_skin(self, sprites, idx=0):
        self.skin = self.sprites[sprites][idx] # переделываем скин из Dict по названию и индексу
        self.body = self.skin.get_rect(topleft=(self.x, self.y)) # переделываем тело по скину

    def blit(self):
        # от сюда удаляем верхнюю строчку, так как она нам больше не нужно
        self.parent.blit(source=self.skin, dest=self.body)
        return self.parent

    def recolor(self, color):...

    def replace(self, x=None, y=None):...

Ну а теперь можно привязывать новый метод к различным действиям<br>
Давайте начнем с метода __motion_control__, в классе __Character__, и  разобъем этот метод на отдельные

In [ ]:
class Character(Object):
    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255), speed=1):...

    def __diagonal_speed(self):...

    def __motion_control(self):...
    
    # принимаем название спрайтов для активности и неактивности, а так же частота обновления кадров
    def motion_left(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        left = self.__motion_control()['left'] # сохраняем значение движения
        if left: # если оно есть, то ...
            self.x += left # изменяем координаты
            self.replace() # перемещаем
            self.side = 'left' # меняем сторону
            if sprites_active: # если есть спррайты, то ...
                self.remake_for_skin(sprites_active, idx=0) # переделываем перса по спрайтам активности
        # иначе если есть скин и нет движение, и есть направление, то ...
        elif sprites_active and not left and self.side == 'left':
            self.remake_for_skin(sprite_inactive, idx=0) # переделываем перса по спрайтам неактивности
        return left

    def motion_right(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        right = self.__motion_control()['right']
        if right:
            self.x += right
            self.replace()
            self.last_action = 'right'
            if sprites_active:
                self.remake_for_skin(sprites_active, idx=0)
        elif sprites_active and not right and self.last_action == 'right':
            self.remake_for_skin(sprite_inactive, idx=0)
        return right

    def motion_up(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        up = self.__motion_control()['up']
        if up:
            self.y += up
            self.replace()
            self.last_action = 'up'
            if sprites_active:
                self.remake_for_skin(sprites_active, idx=0)
        elif sprites_active and not up and self.last_action == 'up':
            self.remake_for_skin(sprite_inactive, idx=0)
        return up

    def motion_down(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        down = self.__motion_control()['down']
        if down:
            self.y += down
            self.replace()
            self.last_action = 'down'
            if sprites_active:
                self.remake_for_skin(sprites_active, idx=0)
        elif sprites_active and not down and self.last_action == 'down':
            self.remake_for_skin(sprite_inactive, idx=0)
        return down

    def change_speed(self, speed):...

Мы облегчили себе задачу по инициализации спрайтов, теперь она выглядит вот так:

In [ ]:
from first_engine.game import Game
from first_engine import objects


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=4)
        hero.load_sprites(name='stand_right', path='./sprites/viking/stand/right')
        hero.load_sprites(name='stand_left', path='./sprites/viking/stand/left')
        hero.load_sprites(name='run_left', path='./sprites/viking/run/left')
        hero.load_sprites(name='run_right', path='./sprites/viking/run/right')

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)
                hero.blit()

                # Выполнение действий
                hero.motion_left(sprites_active='run_left', sprite_inactive='stand_left')
                hero.motion_right(sprites_active='run_right', sprite_inactive='stand_right')

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=600, height=400).run()


Но остальсо последнее - дополнить внутреннию анимацию действий

Для этого нам снова придется вернуться в класс __Object__ и кое что внем дополнить

In [ ]:
class Object:
    sprites = {}
    last_action = 'right'
    time_to_sprite_update = 6 # добвим переменную отвечающую за частоту обновлений кадров (по умолчанию - 6)
    sprite_id = 0 # добавим счетчик спрайтов, начнем с 1го

    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255)):...

    def load_sprites(self, name: str, path: str) -> list[pygame.Surface]:...

    def remake_for_skin(self, name, idx=0):...
    
    def sprite_update(self, name, time_update=6):
        """
        Метод обновляющий спрайты по очереди
        :param name: str - принимает имя спрайтов похраненных в словарь sprites
        :param time_update: int - принимает время до смены спрайта на новый
        :return: int - возвращает id текущего спрайта
        """
        self.time_to_sprite_update -= 1 # отсчет до обновления спрайта
        if self.time_to_sprite_update == 0: # если время вышло
            self.sprite_id += 1 # тогда меняем спрайт
            self.time_to_sprite_update = time_update # и одновляем время
            if self.sprite_id == len(self.sprites[name]): # если спрайт был последним
                self.sprite_id = 0 # то начинаем снова с 1го
        return self.sprite_id

    def blit(self):...

    def recolor(self, color):...

    def replace(self, x=None, y=None):...

В классе __Object__ все подготовили, переходим к классу __Character__

In [ ]:
class Character(Object):
    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255), speed=1):...

    def __diagonal_speed(self):
        speed_xy = (self.speed ** 2 + self.speed ** 2) ** (1 / 2)  # находим длину вектора x + y по пифагору
        speed_xy = self.speed / speed_xy  # находим долю скорости от суммы векторов
        speed_xy *= self.speed  # теперь находим скорость по диагонали
        return speed_xy

    def __motion_control(self):...

    def motion_left(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        left = self.__motion_control()['left']
        if left:
            self.x += left
            self.replace()
            self.last_action = 'left'
            if sprites_active:
                self.sprite_update(name=sprites_active, time_update=time_to_update) # подключаем обновление текущих спрайтов
                # переделываем положение и тело персонажа по очередным спрайтам
                self.remake_for_skin(name=sprites_active, idx=self.sprite_id)
        elif sprite_inactive and not left and self.last_action == 'left':
            self.sprite_update(name=sprite_inactive, time_update=time_to_update) # подключаем обновление текущих спрайтов
            # переделываем положение и тело персонажа по очередным спрайтам
            self.remake_for_skin(name=sprite_inactive, idx=self.sprite_id)
        return left

    def motion_right(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        right = self.__motion_control()['right']
        if right:
            self.x += right
            self.replace()
            self.last_action = 'right'
            if sprites_active:
                self.sprite_update(name=sprites_active, time_update=time_to_update)
                self.remake_for_skin(name=sprites_active, idx=self.sprite_id)
        elif sprite_inactive and not right and self.last_action == 'right':
            self.sprite_update(name=sprite_inactive, time_update=time_to_update)
            self.remake_for_skin(name=sprite_inactive, idx=self.sprite_id)
        return right

    def motion_up(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        up = self.__motion_control()['up']
        if up:
            self.y += up
            self.replace()
            self.last_action = 'up'
            if sprites_active:
                self.sprite_update(name=sprites_active, time_update=time_to_update)
                self.remake_for_skin(name=sprites_active, idx=self.sprite_id)
        elif sprite_inactive and not up and self.last_action == 'up':
            self.sprite_update(name=sprite_inactive, time_update=time_to_update)
            self.remake_for_skin(name=sprite_inactive, idx=self.sprite_id)
        return up

    def motion_down(self, sprites_active=None, sprite_inactive=None, time_to_update=6):
        down = self.__motion_control()['down']
        if down:
            self.y += down
            self.replace()
            self.last_action = 'down'
            if sprites_active:
                self.sprite_update(name=sprites_active, time_update=time_to_update)
                self.remake_for_skin(name=sprites_active, idx=self.sprite_id)
        elif sprite_inactive and not down and self.last_action == 'down':
            self.sprite_update(name=sprite_inactive, time_update=time_to_update)
            self.remake_for_skin(name=sprite_inactive, idx=self.sprite_id)
        return down

    def change_speed(self, speed):...

Вот и все, теперь мы можем очень легко управлять спрайтами из своего __main__ модуля

In [4]:
from first_engine.game import Game
from first_engine import objects


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=4)
        hero.load_sprites(name='stand_right', path='./sprites/viking/stand/right')
        hero.load_sprites(name='stand_left', path='./sprites/viking/stand/left')
        hero.load_sprites(name='run_left', path='./sprites/viking/run/left')
        hero.load_sprites(name='run_right', path='./sprites/viking/run/right')

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(objects=[hero], FPS=60)
                hero.blit()

                # Выполнение действий
                hero.motion_left(sprites_active='run_left', sprite_inactive='stand_left', time_to_update=6)
                hero.motion_right(sprites_active='run_right', sprite_inactive='stand_right', time_to_update=6)

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=600, height=400).run()

# Задание:
- Опишите логику прыжка и падения персонажа